# 🎬 ETL Pipeline - RAW → SILVER (Lakehouse)

---

## 📋 Sumário

Este notebook implementa o **pipeline ETL completo** que transforma dados brutos (RAW) em dados curados (SILVER) no lakehouse.

### Etapas do Pipeline:

1. **📥 EXTRAÇÃO (Extract)**: Leitura dos arquivos CSV da camada RAW
2. **🔄 TRANSFORMAÇÃO (Transform)**: Limpeza, mesclagem e enriquecimento dos dados
3. **📤 CARGA (Load)**: Inserção no banco de dados MySQL (Lakehouse)

---

### 🎯 Objetivo

Criar um **lakehouse populado** com dados de filmes e avaliações, pronto para análises e consultas SQL.

### 📊 Dados de Entrada (RAW):
- `movies_metadata.csv`: ~45K filmes
- `credits.csv`: ~45K registros de elenco/equipe
- `keywords.csv`: ~46K palavras-chave
- `ratings_small.csv`: ~100K avaliações

### 💾 Dados de Saída (SILVER):
- Tabela `movies`: 45.433 filmes
- Tabela `ratings`: 44.989 avaliações válidas

---

## 📦 1. Importação de Bibliotecas

Carregamento das bibliotecas necessárias para o ETL.

In [ ]:
import pandas as pd
import numpy as np
import json
import warnings
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.pool import NullPool
import mysql.connector

warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")
print(f"📌 Pandas versão: {pd.__version__}")
print(f"📌 NumPy versão: {np.__version__}")

---

## 🔧 2. Configuração da Conexão com o Banco

Conexão com o **MySQL Lakehouse** usando SQLAlchemy.

In [ ]:
# Configurações do banco de dados
DB_CONFIG = {
    'host': 'localhost',  # ou 'db' se estiver dentro do container
    'port': 3306,
    'database': 'movies_db',
    'user': 'app_user',
    'password': 'app_password'
}

# String de conexão
connection_string = (
    f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}"
    f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# Criar engine
engine = create_engine(
    connection_string,
    poolclass=NullPool,
    echo=False
)

# Testar conexão
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 'Conexão OK!' as status"))
        print("✅", result.fetchone()[0])
except Exception as e:
    print(f"❌ Erro na conexão: {e}")

---

## 📥 3. FASE 1: EXTRAÇÃO (Extract)

Leitura dos arquivos CSV da camada RAW.

In [ ]:
# Caminhos dos arquivos
DATA_PATH = '../raw/dados_brutos/'

print("📥 Iniciando extração dos dados...\n")

# 1. Movies Metadata
print("📄 Carregando movies_metadata.csv...")
df_movies = pd.read_csv(
    f'{DATA_PATH}movies_metadata.csv',
    low_memory=False,
    dtype={'id': str}
)
print(f"   ✓ {len(df_movies):,} filmes carregados")

# 2. Credits (elenco e equipe)
print("📄 Carregando credits.csv...")
df_credits = pd.read_csv(
    f'{DATA_PATH}credits.csv',
    dtype={'id': str}
)
print(f"   ✓ {len(df_credits):,} registros de créditos carregados")

# 3. Keywords
print("📄 Carregando keywords.csv...")
df_keywords = pd.read_csv(
    f'{DATA_PATH}keywords.csv',
    dtype={'id': str}
)
print(f"   ✓ {len(df_keywords):,} registros de palavras-chave carregados")

# 4. Ratings
print("📄 Carregando ratings_small.csv...")
df_ratings = pd.read_csv(
    f'{DATA_PATH}ratings_small.csv'
)
print(f"   ✓ {len(df_ratings):,} avaliações carregadas")

print("\n✅ Extração concluída com sucesso!")

### 📊 Visualização dos Dados Brutos

In [ ]:
print("🔍 Primeiras linhas - Movies:")
display(df_movies.head(3))

print("\n🔍 Primeiras linhas - Ratings:")
display(df_ratings.head(3))

print("\n📈 Informações dos DataFrames:")
print(f"Movies: {df_movies.shape}")
print(f"Credits: {df_credits.shape}")
print(f"Keywords: {df_keywords.shape}")
print(f"Ratings: {df_ratings.shape}")

---

## 🔄 4. FASE 2: TRANSFORMAÇÃO (Transform)

Aplicação das transformações e limpezas necessárias.

### 🧹 Etapa 1: Limpeza de IDs Inválidos

In [ ]:
print("🧹 Etapa 1: Limpeza de IDs inválidos\n")

# Remover registros com ID inválido (não numérico)
initial_count = len(df_movies)

df_movies = df_movies[pd.to_numeric(df_movies['id'], errors='coerce').notna()]
df_credits = df_credits[pd.to_numeric(df_credits['id'], errors='coerce').notna()]
df_keywords = df_keywords[pd.to_numeric(df_keywords['id'], errors='coerce').notna()]

removed = initial_count - len(df_movies)
print(f"   ✓ {removed} registros com ID inválido removidos")
print(f"   ✓ {len(df_movies):,} filmes restantes")

### 🔗 Etapa 2: Mesclagem dos DataFrames

In [ ]:
print("🔗 Etapa 2: Mesclagem dos DataFrames\n")

# Mesclar movies com credits
df_merged = df_movies.merge(df_credits, on='id', how='left')
print(f"   ✓ Após merge com credits: {df_merged.shape}")

# Mesclar com keywords
df_merged = df_merged.merge(df_keywords, on='id', how='left')
print(f"   ✓ Após merge com keywords: {df_merged.shape}")

# Remover duplicatas
initial = len(df_merged)
df_merged = df_merged.drop_duplicates(subset=['id'], keep='first')
duplicates = initial - len(df_merged)
print(f"   ✓ {duplicates:,} duplicatas removidas")
print(f"   ✓ DataFrame final: {df_merged.shape}")

### 🔢 Etapa 3: Conversão de Tipos de Dados

In [ ]:
print("🔢 Etapa 3: Conversão de tipos de dados\n")

# Converter ID para inteiro
df_merged['id'] = pd.to_numeric(df_merged['id'], errors='coerce').astype('Int64')

# Converter colunas numéricas
numeric_cols = ['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count']
for col in numeric_cols:
    if col in df_merged.columns:
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

# Converter data de lançamento
df_merged['release_date'] = pd.to_datetime(df_merged['release_date'], errors='coerce')

print("   ✓ Tipos de dados convertidos")
print(f"\n📊 Tipos das principais colunas:")
print(df_merged[['id', 'budget', 'revenue', 'runtime', 'release_date']].dtypes)

### 📦 Etapa 4: Extração de Dados JSON

In [ ]:
print("📦 Etapa 4: Extração de dados JSON\n")

def extract_names_from_json(json_str, key='name', max_items=5, separator=', '):
    """Extrai nomes de uma string JSON."""
    if pd.isna(json_str) or json_str == '':
        return ''
    try:
        data = json.loads(json_str.replace("'", '"'))
        if isinstance(data, list):
            names = [item.get(key, '') for item in data[:max_items] if isinstance(item, dict)]
            return separator.join(filter(None, names))
        return ''
    except:
        return ''

# Extrair gêneros
print("   🎭 Extraindo gêneros...")
df_merged['genres'] = df_merged['genres'].apply(extract_names_from_json)

# Extrair elenco (cast) - top 5
print("   🎬 Extraindo elenco principal...")
df_merged['cast'] = df_merged['cast'].apply(extract_names_from_json)

# Extrair equipe (crew) - diretores
print("   🎥 Extraindo equipe técnica...")
df_merged['crew'] = df_merged['crew'].apply(extract_names_from_json)

# Extrair keywords
print("   🔑 Extraindo palavras-chave...")
df_merged['keywords'] = df_merged['keywords'].apply(extract_names_from_json)

# Extrair production companies
print("   🏢 Extraindo produtoras...")
df_merged['production_companies'] = df_merged['production_companies'].apply(extract_names_from_json)

# Extrair production countries
print("   🌍 Extraindo países de produção...")
df_merged['production_countries'] = df_merged['production_countries'].apply(extract_names_from_json)

# Extrair spoken languages
print("   🗣️  Extraindo idiomas...")
df_merged['spoken_languages'] = df_merged['spoken_languages'].apply(extract_names_from_json)

# Extrair belongs_to_collection
print("   📚 Extraindo coleções...")
df_merged['belongs_to_collection'] = df_merged['belongs_to_collection'].apply(
    lambda x: extract_names_from_json(x, key='name', max_items=1) if pd.notna(x) else ''
)

print("\n   ✓ Dados JSON extraídos e processados")

### ✂️ Etapa 5: Seleção de Colunas Finais

In [ ]:
print("✂️ Etapa 5: Seleção de colunas finais\n")

# Colunas para a tabela MOVIES
movies_columns = [
    'id', 'title', 'overview', 'release_date',
    'budget', 'revenue', 'runtime', 'popularity',
    'status', 'tagline', 'vote_average', 'vote_count',
    'imdb_id', 'original_language',
    'genres', 'production_companies', 'production_countries',
    'spoken_languages', 'belongs_to_collection'
]

df_movies_final = df_merged[movies_columns].copy()

# Substituir valores vazios por string vazia
text_columns = ['overview', 'tagline', 'status', 'imdb_id', 'original_language',
                'genres', 'production_companies', 'production_countries',
                'spoken_languages', 'belongs_to_collection']

for col in text_columns:
    df_movies_final[col] = df_movies_final[col].fillna('').astype(str)

print(f"   ✓ DataFrame MOVIES preparado: {df_movies_final.shape}")
print(f"   ✓ Colunas: {len(movies_columns)}")

### 🌟 Etapa 6: Transformação da Tabela RATINGS

In [ ]:
print("🌟 Etapa 6: Transformação da tabela RATINGS\n")

# Renomear colunas
df_ratings_final = df_ratings.rename(columns={
    'userId': 'user_id',
    'movieId': 'movie_id',
    'timestamp': 'rating_timestamp'
})

# Converter timestamp UNIX para datetime
df_ratings_final['rating_timestamp'] = pd.to_datetime(
    df_ratings_final['rating_timestamp'], 
    unit='s'
)

# Remover avaliações de filmes que não existem no dataset final
valid_movie_ids = set(df_movies_final['id'].dropna())
initial_ratings = len(df_ratings_final)
df_ratings_final = df_ratings_final[df_ratings_final['movie_id'].isin(valid_movie_ids)]
removed_ratings = initial_ratings - len(df_ratings_final)

print(f"   ✓ {removed_ratings:,} avaliações de filmes inexistentes removidas")
print(f"   ✓ {len(df_ratings_final):,} avaliações válidas")

# Estatísticas
print(f"\n📊 Estatísticas das avaliações:")
print(f"   • Usuários únicos: {df_ratings_final['user_id'].nunique():,}")
print(f"   • Filmes avaliados: {df_ratings_final['movie_id'].nunique():,}")
print(f"   • Nota média: {df_ratings_final['rating'].mean():.2f}")
print(f"   • Período: {df_ratings_final['rating_timestamp'].min()} a {df_ratings_final['rating_timestamp'].max()}")

### ✅ Resumo da Transformação

In [ ]:
print("\n" + "="*70)
print("✅ TRANSFORMAÇÃO CONCLUÍDA COM SUCESSO!")
print("="*70)
print(f"\n📊 Resumo Final:")
print(f"   • Filmes processados: {len(df_movies_final):,}")
print(f"   • Avaliações processadas: {len(df_ratings_final):,}")
print(f"   • Colunas na tabela MOVIES: {len(df_movies_final.columns)}")
print(f"   • Colunas na tabela RATINGS: {len(df_ratings_final.columns)}")

print(f"\n🔍 Amostra dos dados transformados (MOVIES):")
display(df_movies_final[['id', 'title', 'genres', 'release_date', 'budget', 'revenue']].head(3))

print(f"\n🔍 Amostra dos dados transformados (RATINGS):")
display(df_ratings_final.head(3))

---

## 📤 5. FASE 3: CARGA (Load)

Inserção dos dados no banco de dados MySQL (Lakehouse).

### 🧹 Limpeza das Tabelas Existentes

In [ ]:
print("🧹 Limpando tabelas existentes...\n")

try:
    with engine.connect() as conn:
        # Desabilitar foreign key checks temporariamente
        conn.execute(text("SET FOREIGN_KEY_CHECKS = 0"))
        
        # Truncar tabelas
        conn.execute(text("TRUNCATE TABLE ratings"))
        conn.execute(text("TRUNCATE TABLE movies"))
        
        # Reabilitar foreign key checks
        conn.execute(text("SET FOREIGN_KEY_CHECKS = 1"))
        
        conn.commit()
    
    print("   ✓ Tabelas limpas com sucesso")
except Exception as e:
    print(f"   ⚠️  Aviso: {e}")

### 📥 Carregamento da Tabela MOVIES

In [ ]:
print("\n📥 Carregando tabela MOVIES...\n")

try:
    # Inserir dados em chunks para melhor performance
    chunk_size = 1000
    total_chunks = (len(df_movies_final) // chunk_size) + 1
    
    for i, chunk in enumerate(range(0, len(df_movies_final), chunk_size), 1):
        df_chunk = df_movies_final.iloc[chunk:chunk + chunk_size]
        df_chunk.to_sql(
            name='movies',
            con=engine,
            if_exists='append',
            index=False,
            method='multi',
            chunksize=500
        )
        print(f"   ⏳ Progresso: {i}/{total_chunks} chunks ({(i/total_chunks)*100:.1f}%)", end='\r')
    
    print(f"\n   ✅ {len(df_movies_final):,} filmes carregados com sucesso!")
    
except Exception as e:
    print(f"\n   ❌ Erro ao carregar MOVIES: {e}")

### 📥 Carregamento da Tabela RATINGS

In [ ]:
print("\n📥 Carregando tabela RATINGS...\n")

try:
    # Inserir dados em chunks
    chunk_size = 1000
    total_chunks = (len(df_ratings_final) // chunk_size) + 1
    
    for i, chunk in enumerate(range(0, len(df_ratings_final), chunk_size), 1):
        df_chunk = df_ratings_final.iloc[chunk:chunk + chunk_size]
        df_chunk.to_sql(
            name='ratings',
            con=engine,
            if_exists='append',
            index=False,
            method='multi',
            chunksize=500
        )
        print(f"   ⏳ Progresso: {i}/{total_chunks} chunks ({(i/total_chunks)*100:.1f}%)", end='\r')
    
    print(f"\n   ✅ {len(df_ratings_final):,} avaliações carregadas com sucesso!")
    
except Exception as e:
    print(f"\n   ❌ Erro ao carregar RATINGS: {e}")

---

## ✅ 6. VALIDAÇÃO E ESTATÍSTICAS FINAIS

Verificação dos dados carregados no lakehouse.

In [ ]:
print("\n" + "="*70)
print("✅ PIPELINE ETL CONCLUÍDO COM SUCESSO!")
print("="*70)

# Verificar contadores no banco
print("\n📊 Verificando dados no lakehouse...\n")

with engine.connect() as conn:
    # Contar filmes
    result = conn.execute(text("SELECT COUNT(*) FROM movies"))
    movies_count = result.fetchone()[0]
    print(f"   🎬 Filmes na tabela MOVIES: {movies_count:,}")
    
    # Contar avaliações
    result = conn.execute(text("SELECT COUNT(*) FROM ratings"))
    ratings_count = result.fetchone()[0]
    print(f"   ⭐ Avaliações na tabela RATINGS: {ratings_count:,}")
    
    # Usuários únicos
    result = conn.execute(text("SELECT COUNT(DISTINCT user_id) FROM ratings"))
    users_count = result.fetchone()[0]
    print(f"   👥 Usuários únicos: {users_count:,}")
    
    # Estatísticas gerais
    print("\n📈 Executando procedure de estatísticas...\n")
    result = conn.execute(text("CALL sp_database_stats()"))
    stats = result.fetchall()
    
    for row in stats:
        print(f"\n   📊 Tabela: {row[0]}")
        print(f"      • Total de registros: {row[1]:,}")
        print(f"      • IDs únicos: {row[2]:,}")
        print(f"      • Data mais antiga: {row[3]}")
        print(f"      • Data mais recente: {row[4]}")
        if row[5] is not None:
            print(f"      • Orçamento médio: ${row[5]:,.2f}")
        if row[6] is not None:
            print(f"      • Receita média: ${row[6]:,.2f}")
        if row[7] is not None:
            print(f"      • Avaliação média: {row[7]:.2f}")

print("\n" + "="*70)
print("🎉 LAKEHOUSE POPULADO E PRONTO PARA USO!")
print("="*70)

---

## 🔍 7. CONSULTAS DE EXEMPLO

Demonstração de consultas SQL usando as views criadas.

### 📊 View: Filmes com Estatísticas Agregadas

In [ ]:
# Top 10 filmes mais lucrativos
query = """
SELECT 
    title,
    genres,
    YEAR(release_date) as ano,
    budget,
    revenue,
    roi_percentage,
    user_avg_rating,
    user_ratings_count
FROM v_movies_with_stats
WHERE revenue > 0 AND budget > 0
ORDER BY revenue DESC
LIMIT 10
"""

df_top_revenue = pd.read_sql(query, engine)
print("🏆 Top 10 Filmes Mais Lucrativos:\n")
display(df_top_revenue)

### 🎭 View: Distribuição por Gênero

In [ ]:
query = """
SELECT *
FROM v_genre_distribution
ORDER BY total_movies DESC
LIMIT 10
"""

df_genres = pd.read_sql(query, engine)
print("🎭 Top 10 Gêneros Mais Populares:\n")
display(df_genres)

### 📅 View: Melhores Filmes por Ano

In [ ]:
query = """
SELECT *
FROM v_top_movies_by_year
WHERE release_year >= 2010
ORDER BY release_year DESC, user_avg_rating DESC
LIMIT 15
"""

df_top_by_year = pd.read_sql(query, engine)
print("📅 Melhores Filmes por Ano (2010+):\n")
display(df_top_by_year)

---

## 🎯 CONCLUSÃO

### ✅ Objetivos Alcançados:

1. **Extração**: Leitura de 4 arquivos CSV (~237K registros)
2. **Transformação**: 
   - Limpeza de IDs inválidos
   - Mesclagem de múltiplos datasets
   - Conversão de tipos de dados
   - Extração de dados JSON complexos
   - Normalização e validação
3. **Carga**: Inserção de 45K+ filmes e 45K+ avaliações no MySQL

### 📊 Estatísticas Finais:
- **Filmes**: 45.433
- **Avaliações**: 44.989
- **Usuários**: 671
- **Período**: 1995-2016

### 🚀 Próximos Passos:
- Camada GOLD: Star Schema para Data Warehouse
- Dashboards no Power BI/Tableau
- Análises avançadas e Machine Learning

---

**🎬 Lakehouse pronto para análises!**